In [1]:
import pandas as pd
import numpy as np
from scipy import stats

## データの読み込み、準備

In [2]:
url = 'http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv'
df = pd.read_csv(url)

In [3]:
df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


女性向けデータの削除、介入を表す列（treatment）の追加

- 簡単のため。女性へのメール配信のデータを削除

In [4]:
male_df = df.loc[df.segment != 'Womens E-Mail'].copy()
male_df['treatment'] = 0
male_df.loc[male_df.segment == 'Mens E-Mail', 'treatment'] = 1

介入の有無に応じた集計を確認

In [5]:
summary_by_segment = pd.merge(
                        male_df.groupby('treatment')[['conversion', 'spend']].mean().reset_index(), 
                        male_df.groupby('treatment').size().reset_index(),
                        on='treatment'
                    ).rename(columns = {'conversion': 'conversion_mean', 'spend': 'spend_mean', 0:'count'})
print(summary_by_segment)

   treatment  conversion_mean  spend_mean  count
0          0         0.005726    0.652789  21306
1          1         0.012531    1.422617  21307


- 男性向けメールが配信されたグループでは、conversionが発生する確率が1.25%、一方、メールが配信されなかったグループは0.573%で購買が発生する確率が買いことが分かる
- 売上金額についても、平均するとメールが配信されたグループの方が高い
- すなわち、メールによりこう平均売上金額が上がっていることが分かる

## 有意差検定

In [6]:
mens_mail = np.array(male_df.loc[male_df['treatment'] == 1]['spend'])
no_mail = np.array(male_df.loc[male_df['treatment'] == 0]['spend'])

対応の無いデータに対するt検定。Welchのt検定を行うので、ttest_indメソッドでequal_val=Falseとする

In [7]:
stats.ttest_ind(mens_mail, no_mail, equal_var=False)

Ttest_indResult(statistic=5.300140358411668, pvalue=1.1638149682254859e-07)

- pvalue=1.1638149682254859e-07と5%以下で有意であることが分かる

## 1.4.3 バイアスのあるデータの検証

In [17]:
# 購買する傾向が高そうなサンプルの割合を増やし、購買しない傾向が高そうなサンプルの割合は減らす
# 購買する傾向が高そうなサンプル: sample_rules
sample_rules = (male_df.history>300) | (male_df.recency<6) | (male_df.channel=="Multichannel")
biased_data = pd.concat([
    male_df[(sample_rules)&(male_df.treatment==0)].sample(frac=0.5, random_state=1),
    male_df[(sample_rules)&(male_df.treatment==1)],
    male_df[~(sample_rules)&(male_df.treatment==0)],
    male_df[~(sample_rules)&(male_df.treatment==1)].sample(frac=0.5, random_state=1),
], axis=0, ignore_index=True)
pd.merge(
    biased_data.groupby('treatment')[['conversion', 'spend']].mean().reset_index(), 
    biased_data.groupby('treatment').size().reset_index(),
    on='treatment'
).rename(columns = {'conversion': 'conversion_mean', 'spend': 'spend_mean', 0:'count'})

,treatment,conversion_mean,spend_mean,count
0,0,0.004540,0.557954,14757
1,1,0.013572,1.541704,17168


- 二週間以内の購入確率、購買金額の差がともに大きくなっている

In [21]:
biased_mail = np.array(biased_data.loc[biased_data['treatment'] == 1]['spend'])
biased_no_mail = np.array(biased_data.loc[biased_data['treatment'] == 0]['spend'])
stats.ttest_ind(biased_mail, biased_no_mail, equal_var=False)

Ttest_indResult(statistic=5.799473880787556, pvalue=6.722150409691523e-09)

- バイアスのあるデータをt検定した結果、p値はますます小さくなっている
- このことから、統計的な優位が正しい効果の推定が得られるわけではないことが分かる